# Лабораторная работа №1

Борисов Дмитрий (6133-010402D)

**Задачи**:
1) Найти велосипед с максимальным пробегом.

2) Найти наибольшее расстояние между станциями.

3) Найти путь велосипеда с максимальным пробегом через станции.

4) Найти количество велосипедов в системе.

5) Найти пользователей потративших на поездки более 3 часов.

In [1]:
!pip install pyspark

In [2]:
from pyspark import SparkContext, SparkConf 

In [3]:
conf = SparkConf().setAppName("Lab1_BigData").setMaster('local[1]')
sc = SparkContext(conf=conf)

In [4]:
sc

<SparkContext master=local[1] appName=Lab1_BigData>

### Считывание файлов

In [5]:
trips = sc.textFile("/mnt/data/trips.csv")
trips.take(10)

['4576,63,2013-08-29 14:13,South Van Ness at Market,66,2013-08-29 14:14,South Van Ness at Market,66,520,Subscriber,94127',
 '4607,70,2013-08-29 14:42,San Jose City Hall,10,2013-08-29 14:43,San Jose City Hall,10,661,Subscriber,95138',
 '4130,71,2013-08-29 10:16,Mountain View City Hall,27,2013-08-29 10:17,Mountain View City Hall,27,48,Subscriber,97214',
 '4251,77,2013-08-29 11:29,San Jose City Hall,10,2013-08-29 11:30,San Jose City Hall,10,26,Subscriber,95060',
 '4299,83,2013-08-29 12:02,South Van Ness at Market,66,2013-08-29 12:04,Market at 10th,67,319,Subscriber,94103',
 '4927,103,2013-08-29 18:54,Golden Gate at Polk,59,2013-08-29 18:56,Golden Gate at Polk,59,527,Subscriber,94109',
 '4500,109,2013-08-29 13:25,Santa Clara at Almaden,4,2013-08-29 13:27,Adobe on Almaden,5,679,Subscriber,95112',
 '4563,111,2013-08-29 14:02,San Salvador at 1st,8,2013-08-29 14:04,San Salvador at 1st,8,687,Subscriber,95112',
 '4760,113,2013-08-29 17:01,South Van Ness at Market,66,2013-08-29 17:03,South Van Ne

In [6]:
stations = sc.textFile("/mnt/data/stations.csv")
stations.take(10)

['2,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,2013-08-06,',
 '3,San Jose Civic Center,37.330698,-121.888979,15,San Jose,2013-08-05,',
 '4,Santa Clara at Almaden,37.333988,-121.894902,11,San Jose,2013-08-06,',
 '5,Adobe on Almaden,37.331415,-121.8932,19,San Jose,2013-08-05,',
 '6,San Pedro Square,37.336721,-121.894074,15,San Jose,2013-08-07,',
 '7,Paseo de San Antonio,37.333798,-121.886943,15,San Jose,2013-08-07,',
 '8,San Salvador at 1st,37.330165,-121.885831,15,San Jose,2013-08-05,',
 '9,Japantown,37.348742,-121.894715,15,San Jose,2013-08-05,',
 '10,San Jose City Hall,37.337391,-121.886995,15,San Jose,2013-08-06,',
 '11,MLK Library,37.335885,-121.88566,19,San Jose,2013-08-06,']

### Подготовка данных

In [7]:
trips_data = trips.map(lambda x: x.split(','))
stations_data = stations.map(lambda x: x.split(','))

In [8]:
trips_data.take(3)

[['4576',
  '63',
  '2013-08-29 14:13',
  'South Van Ness at Market',
  '66',
  '2013-08-29 14:14',
  'South Van Ness at Market',
  '66',
  '520',
  'Subscriber',
  '94127'],
 ['4607',
  '70',
  '2013-08-29 14:42',
  'San Jose City Hall',
  '10',
  '2013-08-29 14:43',
  'San Jose City Hall',
  '10',
  '661',
  'Subscriber',
  '95138'],
 ['4130',
  '71',
  '2013-08-29 10:16',
  'Mountain View City Hall',
  '27',
  '2013-08-29 10:17',
  'Mountain View City Hall',
  '27',
  '48',
  'Subscriber',
  '97214']]

In [9]:
stations_data.take(3)

[['2',
  'San Jose Diridon Caltrain Station',
  '37.329732',
  '-121.901782',
  '27',
  'San Jose',
  '2013-08-06',
  ''],
 ['3',
  'San Jose Civic Center',
  '37.330698',
  '-121.888979',
  '15',
  'San Jose',
  '2013-08-05',
  ''],
 ['4',
  'Santa Clara at Almaden',
  '37.333988',
  '-121.894902',
  '11',
  'San Jose',
  '2013-08-06',
  '']]

In [10]:
from typing import NamedTuple
from datetime import datetime

In [11]:
class Station(NamedTuple):
    station_id: int
    name: str
    lat: float
    long: float
    dockcount: int
    landmark: str
    installation: str
    notes: str

    def init(station):
        return Station(
            station_id = int(station[0]),
            name = station[1],
            lat = float(station[2]),
            long = float(station[3]),
            dockcount = int(station[4]),
            landmark = station[5],
            installation = station[6],
            notes = station[7]
        )

In [12]:
class Trip(NamedTuple):
    trip_id: int
    duration: int
    start_date: datetime
    start_station: str
    start_terminal: int
    end_date: datetime
    end_station: str
    end_terminal: int
    bike_id: int
    subscription_type: str
    zip_code: str
        
    def init(trip):
        return Trip(                             
             trip_id = int(trip[0]),
             duration = int(trip[1]),
             start_date = datetime.strptime(trip[2], '%Y-%m-%d %H:%M'),
             start_station = trip[3],
             start_terminal = int(trip[4]),
             end_date = datetime.strptime(trip[5], '%Y-%m-%d %H:%M'),
             end_station = trip[6],
             end_terminal = trip[7],
             bike_id = int(trip[8]),
             subscription_type = trip[9],
             zip_code = trip[10]
        )

In [13]:
trips = trips_data.map(lambda row: Trip.init(row))

In [14]:
trips.first()

Trip(trip_id=4576, duration=63, start_date=datetime.datetime(2013, 8, 29, 14, 13), start_station='South Van Ness at Market', start_terminal=66, end_date=datetime.datetime(2013, 8, 29, 14, 14), end_station='South Van Ness at Market', end_terminal='66', bike_id=520, subscription_type='Subscriber', zip_code='94127')

In [15]:
stations = stations_data.map(lambda row: Station.init(row))

In [16]:
stations.first()

Station(station_id=2, name='San Jose Diridon Caltrain Station', lat=37.329732, long=-121.901782, dockcount=27, landmark='San Jose', installation='2013-08-06', notes='')

### Задание 1. Найти велосипед с максимальным пробегом.

#### Так как у велосипеда явно не указан пройденный путь, то в качестве пробега возьмём продолжительность (duration) пути

In [17]:
bike_and_trip = trips.keyBy(lambda trip: trip.bike_id)
bike_and_trip.first()

(520,
 Trip(trip_id=4576, duration=63, start_date=datetime.datetime(2013, 8, 29, 14, 13), start_station='South Van Ness at Market', start_terminal=66, end_date=datetime.datetime(2013, 8, 29, 14, 14), end_station='South Van Ness at Market', end_terminal='66', bike_id=520, subscription_type='Subscriber', zip_code='94127'))

In [18]:
group_by_bike = bike_and_trip\
                .mapValues(lambda trip: trip.duration)\
                .reduceByKey(lambda duration1, duration2: duration1 + duration2)
group_by_bike.first()

(520, 844850)

In [19]:
max_duration_of_bike = group_by_bike.reduce(lambda bike1, bike2: bike1 if bike1[1] > bike2[1] else bike2)
max_duration_of_bike

(465, 1402501)

In [20]:
bike_id_with_max_duration = max_duration_of_bike[0]
print(f"Bike with max duration is {bike_id_with_max_duration}")

Bike with max duration is 465


### Задание 2. Найти наибольшее расстояние между станциями.

#### Расстоянием между двумя станциями будем считать расстояние между координатами станций на земной поверхности. Таким образом, если найти соотнести каждую станцию с каждой (декартово произведение), то по полученным двум точкам можно найти расстояние воспользовавшись формулой.

In [21]:
vectors = stations.cartesian(stations)\
                .filter(lambda vector: vector[0].station_id != vector[1].station_id)\
                .keyBy(lambda vector: (vector[0].name, vector[1].name))
vectors.take(3)               

[(('San Jose Diridon Caltrain Station', 'San Jose Civic Center'),
  (Station(station_id=2, name='San Jose Diridon Caltrain Station', lat=37.329732, long=-121.901782, dockcount=27, landmark='San Jose', installation='2013-08-06', notes=''),
   Station(station_id=3, name='San Jose Civic Center', lat=37.330698, long=-121.888979, dockcount=15, landmark='San Jose', installation='2013-08-05', notes=''))),
 (('San Jose Diridon Caltrain Station', 'Santa Clara at Almaden'),
  (Station(station_id=2, name='San Jose Diridon Caltrain Station', lat=37.329732, long=-121.901782, dockcount=27, landmark='San Jose', installation='2013-08-06', notes=''),
   Station(station_id=4, name='Santa Clara at Almaden', lat=37.333988, long=-121.894902, dockcount=11, landmark='San Jose', installation='2013-08-06', notes=''))),
 (('San Jose Diridon Caltrain Station', 'Adobe on Almaden'),
  (Station(station_id=2, name='San Jose Diridon Caltrain Station', lat=37.329732, long=-121.901782, dockcount=27, landmark='San Jose'

In [22]:
!pip install numpy

In [23]:
import numpy as np

Для вычисления расстояния между точками на поверхности земного шара можно использовать формулу, известную в сферической геометрии и геодезии:

S = 111,2×arccos(sin φ1 × sin φ2 + cos φ1 × cos φ2 × cos (L2-L1)),

где S - расстояние, км;

φ1 и L1 - широта и долгота точки 1 (для северной широты и восточной долготы со знаком плюс, для южной широты и западной долготы со знаком минус), градусы;

φ2 и L2 - широта и долгота точки 2, градусы;

константа 111,2 - средняя длина дуги в один градус на поверхности Земли, км.

source: http://mk.semico.ru/dr_info19.htm

In [24]:
def geometry_lenght(phi1, L1, phi2, L2):
    return 111.2 * np.arccos(np.sin(phi1) * np.sin(phi2) + np.cos(phi1)*np.cos(phi2)*np.cos(L2-L1))

In [25]:
square_lenghts = vectors\
            .mapValues(lambda vector: geometry_lenght(vector[0].lat, vector[0].long, vector[1].lat, vector[1].long))
square_lenghts.take(5)

[(('San Jose Diridon Caltrain Station', 'San Jose Civic Center'),
  1.3322520062755303),
 (('San Jose Diridon Caltrain Station', 'Santa Clara at Almaden'),
  0.8566410691824484),
 (('San Jose Diridon Caltrain Station', 'Adobe on Almaden'),
  0.9096990716666701),
 (('San Jose Diridon Caltrain Station', 'San Pedro Square'),
  1.1156299005714054),
 (('San Jose Diridon Caltrain Station', 'Paseo de San Antonio'),
  1.60500605931874)]

In [26]:
max_square_lenght = square_lenghts.reduce(lambda len1, len2: len1 if len1[1] > len2[1] else len2)
max_square_lenght

(('Embarcadero at Sansome', 'SJSU - San Salvador at 9th'), 77.69328061354797)

In [27]:
print(f"Max distance between stations is {max_square_lenght[1]} km")

Max distance between stations is 77.69328061354797 km


### Задание 3. Найти путь велосипеда с максимальным пробегом через станции.

#### Так как id велосипеда с максимальным пробегом уже известен из задания 1, то можно взять данные по этому велосипеду и восстановить хронологию его поездок.

In [28]:
bike_path = trips.filter(lambda trip: trip.bike_id == bike_id_with_max_duration)\
        .sortBy(lambda trip: trip.start_date)\
        .map(lambda trip: (trip.start_station, trip.end_station))

In [29]:
bike_path.collect()

[('Market at 10th', 'Market at 4th'),
 ('Market at 4th', 'Harry Bridges Plaza (Ferry Building)'),
 ('Harry Bridges Plaza (Ferry Building)', 'Howard at 2nd'),
 ('Howard at 2nd', 'Townsend at 7th'),
 ('Townsend at 7th', 'Steuart at Market'),
 ('Steuart at Market', '2nd at Townsend'),
 ('2nd at Townsend', 'Clay at Battery'),
 ('Clay at Battery', '2nd at Townsend'),
 ('2nd at Townsend', 'South Van Ness at Market'),
 ('South Van Ness at Market', 'Powell Street BART'),
 ('Powell Street BART', 'South Van Ness at Market'),
 ('South Van Ness at Market', 'Powell at Post (Union Square)'),
 ('Powell at Post (Union Square)', 'San Francisco Caltrain (Townsend at 4th)'),
 ('San Francisco Caltrain (Townsend at 4th)', 'Post at Kearney'),
 ('Post at Kearney', 'Embarcadero at Sansome'),
 ('Embarcadero at Sansome', '2nd at Townsend'),
 ('2nd at Townsend', 'Market at Sansome'),
 ('Market at Sansome', 'South Van Ness at Market'),
 ('South Van Ness at Market', 'Civic Center BART (7th at Market)'),
 ('Civic C

### Задание 4. Найти количество велосипедов в системе.

#### Выводим список уникальных id велосипедов и считаем их количество

In [30]:
bikes = trips.map(lambda trip: trip.bike_id).distinct().count()
bikes

699

### Задание 5. Найти пользователей потративших на поездки более 3 часов.

#### Будем считать идентификатор поездки, как отдельного пользователя

In [31]:
three_hour = 3 * 60 * 60    # sec
users = trips.filter(lambda user: user.duration > three_hour).map(lambda user: user.trip_id)

In [32]:
users.take(10)

[4639, 4637, 4528, 4363, 4193, 4190, 4225, 4663, 4532, 4521]

In [33]:
users.count()

4770